El club te ha proporcionado un dataset con las métricas físicas y técnicas de la plantilla. Tu objetivo es realizar un Análisis Exploratorio de Datos (EDA) completo para identificar patrones, evaluar la consistencia de los jugadores y entender qué variables físicas influyen en el rendimiento goleador.

Variables a analizar:

+ Goles: Rendimiento final.
+ Minutos Jugados: Consistencia y presencia.
+ Velocidad Máxima (km/h): Potencia física.
+ Distancia Recorrida (km): Resistencia.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Generación de datos simulados
np.random.seed(42)
n_jugadores = 25

data = {
    'Nombre': [f'Jugador {i+1}' for i in range(n_jugadores)],
    'Goles': np.random.poisson(8, n_jugadores),
    'Minutos': np.random.normal(2000, 500, n_jugadores).clip(500, 3000),
    'Velocidad_Max': np.random.normal(31, 2.5, n_jugadores),
    'Distancia_km': np.random.normal(10, 1.5, n_jugadores)
}

df_futbol = pd.DataFrame(data)
df_futbol.head()

,Nombre,Goles,Minutos,Velocidad_Max,Distancia_km
0,Jugador 1,6,2087.416506,33.442950,10.456674
1,Jugador 2,7,1391.572554,31.104950,10.385811
2,Jugador 3,6,2524.673696,30.656820,10.035898
3,Jugador 4,7,2662.552831,30.689669,11.307871
4,Jugador 5,7,2367.250532,32.850852,12.156034


# 1
Analizamos si los goles están repartidos o concentrados en pocos jugadores.

Crea un histograma. El eje X son los goles. La altura de la barra (eje y) es el número de jugadores con esos goles.

+ Pinta línea vertical en rojo la media y mediana.
+ Pinta una zona rojo semi transparence con 1sigma.

Pinta la leyenda donde se vean los valores (media, mediana y desviación)

In [ ]:
# 1. Configuración de estilo
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))

# 2. Cálculo de métricas
media = df_futbol['Goles'].mean()
mediana = df_futbol['Goles'].median()
std_dev = df_futbol['Goles'].std()

# 3. Creación del Histograma
n, bins, patches = plt.hist(df_futbol['Goles'], bins=10, color='skyblue', edgecolor='black', alpha=0.7)

# 4. Líneas verticales (Media y Mediana)
plt.axvline(media, color='red', linestyle='--', linewidth=2, label=f'Media: {media:.2f}')
plt.axvline(mediana, color='darkred', linestyle='-', linewidth=2, label=f'Mediana: {mediana:.2f}')

# 5. Sombreado de Desviación Estándar (1 Sigma)
plt.axvspan(media - std_dev, media + std_dev, color='red', alpha=0.15, label=f'Desv. Est. (σ): {std_dev:.2f}')

# 6. Estética y etiquetas
plt.title('Distribución de Goles en la Plantilla', fontsize=15)
plt.xlabel('Número de Goles', fontsize=12)
plt.ylabel('Número de Jugadores', fontsize=12)
plt.legend(loc='upper right', frameon=True, shadow=True)

plt.show()

# 2

Comparamos cómo varían las variables en un Boxplot.

In [ ]:
# Configuración del lienzo
plt.figure(figsize=(10, 6))

# Creación del Boxplot comparativo
# Seleccionamos las columnas físicas para que la escala sea legible
sns.boxplot(data=df_futbol[['Velocidad_Max', 'Distancia_km']], palette="Set2")

# Añadir "swarplot" para ver los puntos individuales de cada jugador
sns.swarmplot(data=df_futbol[['Velocidad_Max', 'Distancia_km']], color=".25", alpha=0.6)

# Estética
plt.title('Comparación de Atributos Físicos: Velocidad vs Distancia', fontsize=15)
plt.ylabel('Valor', fontsize=12)
plt.xticks([0, 1], ['Velocidad Máxima (km/h)', 'Distancia Media (km)'])

plt.show()

# 3
Correlación y Relación Visual (Subplots de Scatter)

¿Los que corren más distancia marcan más goles? ¿O es la velocidad? ¿o los minutos jugados?

Analiza en subplots de 3 gráficas los pares goles-velocidad y goles-km y goles-minutos. Goles en el eje y.

Puedes usar sns.scatterplot o plt.scatter que son nubes de puntos simples,  o  mejor sns.regplot que pinta recta de regresión y zona de error.

In [ ]:
# 1. Configuración de la figura con 3 subplots
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)
fig.suptitle('¿Qué factor influye más en los Goles?', fontsize=18, fontweight='bold', y=1.05)

# 2. Goles vs Velocidad Máxima
sns.regplot(ax=axes[0], x='Velocidad_Max', y='Goles', data=df_futbol, 
            color='teal', scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
axes[0].set_title('Velocidad vs Goles')
axes[0].set_xlabel('Velocidad Máxima (km/h)')

# 3. Goles vs Distancia Recorrida
sns.regplot(ax=axes[1], x='Distancia_km', y='Goles', data=df_futbol, 
            color='darkorange', scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
axes[1].set_title('Distancia (km) vs Goles')
axes[1].set_xlabel('Distancia Recorrida (km)')

# 4. Goles vs Minutos Jugados
sns.regplot(ax=axes[2], x='Minutos', y='Goles', data=df_futbol, 
            color='mediumpurple', scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
axes[2].set_title('Minutos vs Goles')
axes[2].set_xlabel('Minutos Jugados')

plt.tight_layout()
plt.show()

# 4
Calcula la matriz de correlación con Pandas y píntala con un heatmap de seaborn


In [ ]:
# 1. Calcular la matriz de correlación (solo columnas numéricas)
# Eliminamos 'Nombre' ya que es texto
corr_matrix = df_futbol.drop(columns=['Nombre']).corr()

# 2. Configuración del estilo
plt.figure(figsize=(10, 8))

# 3. Creación del Heatmap
# annot=True muestra los números, cmap='coolwarm' ayuda a ver extremos
sns.heatmap(corr_matrix, 
            annot=True, 
            fmt=".2f", 
            cmap='coolwarm', 
            linewidths=0.5, 
            center=0,
            cbar_kws={'label': 'Grado de Correlación'})

# 4. Estética final
plt.title('Mapa de Calor: Correlaciones en el Rendimiento', fontsize=16, pad=20)
plt.xticks(rotation=45)
plt.yticks(rotation=0)

plt.show()

# Responde sobre todo lo anterior

+ Forma de la Distribución: ¿El histograma de goles es simétrico o tiene sesgo? (Si la media > mediana, hay goleadores estrella que suben el promedio).

+ Outliers: ¿Hay algún jugador con una velocidad muy por debajo o por encima del resto en el Boxplot?

+ Correlación: ¿Qué variable tiene más peso para marcar goles? 

+ Dispersión: Observando el rango sombreado (±1σ) en el histograma, ¿qué porcentaje de la plantilla se considera "normal" en su rendimiento?